# cancer_maintrain_ensemble

> Basic extensible API to train ensemble. Heavily builds on `cancer_maintrain`

In [ ]:
#| default_exp cancer_maintrain_ensemble

Setup: Surely there is a way to get rid of having to put this cell everywhere. hmmm.

Or we can just copy paste / delete this in and out when needed. Either way, getting close to a decent workable workflow.

In [ ]:
#| hide

def colab_is_true():

    try: 
        from google.colab import drive

        return True 
    except ModuleNotFoundError:
        return False

def setup_colab():
    import os
    from google.colab import drive
    drive.mount('/content/drive',force_remount=True)
    #os.system('unzip -q "/content/drive/My Drive/archive (1).zip"')
    os.system('git clone https://github.com/hamish-haggerty/cancer-proj.git')
    os.chdir('cancer-proj')
    os.system('unzip -q "/content/drive/My Drive/archive (1).zip"') #does this work?
    os.system('pip install -e .')
    os.system('pip install -qU nbdev')
    os.system('nbdev_install_quarto')

if __name__ == "__main__":
    on_colab = colab_is_true()
    if on_colab:
        setup_colab()

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from fastai.vision.all import *
from base_rbt.all import *
from cancer_proj.cancer_dataloading import *
from cancer_proj.cancer_metrics import *
from cancer_proj.cancer_maintrain import *

/Users/hamishhaggerty/opt/anaconda3/envs/thesis/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load the data

In [ ]:
#| hide

#Since we have cloned repository and cd'd into it (and the data itself is not stored in the
#repo) we need cd out of it, get the data, then cd back into the repo `cancer-proj`.
#This is a bit annoying, can maybe remove this later
if on_colab:
    #os.chdir('..') #assumes we are currently in cancer-proj directory
    train_dir = colab_train_dir
    test_dir = colab_test_dir
else:
    train_dir = local_train_dir
    test_dir = local_test_dir

#define general hps
device ='cuda' if torch.cuda.is_available() else 'cpu'
#bs=256
#bs=698
bs=256
bs_tune=256
size=128
bs_val=174

#get the data dictionary
data_dict = get_fnames_dls_dict(train_dir=train_dir,test_dir=test_dir,
                    device=device,bs_val=bs_val,bs=bs,bs_tune=bs_tune,size=size,n_in=3)

#get the dataloaders
dls_train,dls_tune,dls_valid = data_dict['dls_train'],data_dict['dls_tune'],data_dict['dls_valid']
x,y = data_dict['x'],data_dict['y']
xval,yval = data_dict['xval'],data_dict['yval']
xtune,ytune = data_dict['xtune'],data_dict['ytune']
vocab = data_dict['vocab']

#If we want to write some tests (make sure the data is same every time etc):
fnames,fnames_train,fnames_tune,fnames_valid,fnames_test = data_dict['fnames'],data_dict['fnames_train'],data_dict['fnames_tune'],data_dict['fnames_valid'],data_dict['fnames_test']

test_eq(x.shape,xtune.shape)

# if on_colab:
#     os.chdir('cancer-proj')

## Load aug pipelines here

In [ ]:
#| hide

aug_dict = create_aug_pipelines(size=size,device=device,Augs=BYOL_Augs,TUNE_Augs=TUNE_Augs,Val_Augs=Val_Augs)
aug_pipelines = aug_dict['aug_pipelines']
aug_pipelines_tune = aug_dict['aug_pipelines_tune']
aug_pipelines_test = aug_dict['aug_pipelines_test'] 

In [ ]:
#| export

class P4BarlowTwinsModel(Module):

    def __init__(self,model,encoder2,projector2):
        self.model = model #frozen model
        self.encoder2 = encoder2
        self.projector2 = projector2

        #put on GPU
        if torch.cuda.is_available(): self.to(torch.device('cuda'))

        
    def forward(self,x,y):
        
        enc = self.model.encoder(x)
        enc2 = self.encoder2(x)

        return self.model.projector(enc),self.projector2(enc2),enc,enc2


In [ ]:
#| export

def create_p4barlow_twins_model(model,encoder2, hidden_size=256, projection_size=128, bn=True, nlayers=3):
    "Create Barlow Twins model"
    n_in  = in_channels(encoder2)
    with torch.no_grad(): 
        model.cpu()
        encoder2.cpu()
        
        representation = encoder2(torch.randn((2,n_in,128,128)))

    model.cpu()
    encoder2.cpu()

    projector2 = create_mlp_module(representation.size(1), hidden_size, projection_size, bn=bn, nlayers=nlayers) 
    apply_init(projector2)

    return P4BarlowTwinsModel(model=model,encoder2=encoder2,projector2=projector2)


In [ ]:
#| export

def show_btens_batch(dls,n_in,aug,n=2,print_augs=True):
    "Given a linear learner, show a batch"
        
    learn = Learner(dls,model=None, cbs=[BarlowTwinsEns(aug,n_in=n_in,print_augs=print_augs)])
    b = dls.one_batch()
    learn._split(b)
    learn('before_batch')
    axes = learn.barlow_twins_ens.show(n=n)

In [ ]:
#| export

def create_ens_model(model,device,ps=8192,n_in=3):
    "Input a barlow twins model (encoder and projector) that has already been trained"

    #Put into eval mode and turn off gradient
    model.eval()
    model = grad_on(model,on=False)

    _,encoder2 = create_model('no_pretrain',device,ps=ps,n_in=3)
    model2 = create_p4barlow_twins_model(model,encoder2,hidden_size=ps,projection_size=ps)

    encoder2 = model2.encoder2
    if device == 'cuda':
        model2.cuda()
        encoder2.cuda()


    return model2,encoder2

In [ ]:
#| export
def cov_mat(Z1,Z2):
    "Generally, we assume that has already been normalized"

    bs=Z1.shape[0]

    return (Z1.T @ Z2) / bs


There are two kinds of loss functions (type 1 and type 2). Each of these can be further computed in encoder or projector space. Type 1 is closer to standard RR.

In [ ]:
#| export

    
def lf_1_enc(pred,I,lmb,
               t,
               s,
               ):
    
    "compute `standard` rr in proj-encoder space"
    pred1 = pred[0] #frozen
    pred2 = pred[1] #has gradients

    pred1_enc = pred[2]
    pred2_enc = pred[3]
    
    
    bs,nf = pred1.size(0)//2,pred1.size(1)

    #All standard, from BT
    z1, z2 = pred1[:bs],pred1[bs:] #so z1 is bs*projection_size, likewise for z2
    z1norm = (z1 - z1.mean(0)) / (z1.std(0, unbiased=False) + 1e-7)
    z2norm = (z2 - z2.mean(0)) / (z2.std(0, unbiased=False) + 1e-7)
    
    z1_2, z2_2 = pred2[:bs],pred2[bs:] #so z1 is bs*projection_size, likewise for z2
    z1norm_2 = (z1_2 - z1_2.mean(0)) / (z1_2.std(0, unbiased=False) + 1e-7)
    z2norm_2 = (z2_2 - z2_2.mean(0)) / (z2_2.std(0, unbiased=False) + 1e-7)

    
    z1, z2 = pred1_enc[:bs],pred1_enc[bs:] #so z1 is bs*projection_size, likewise for z2
    z1norm_enc = (z1 - z1.mean(0)) / (z1.std(0, unbiased=False) + 1e-7)
    z2norm_enc = (z2 - z2.mean(0)) / (z2.std(0, unbiased=False) + 1e-7)

    #We don't actually need encoder for unfrozen model, I believe, at this point.
    
    #Make sure gradients are turned off / turned on appropriately
    test_eq(pred1.requires_grad,False)
    test_eq(pred2.requires_grad,True)

    # #Within frozen model 
    # C = (z1norm.T @ z2norm) / bs 
    # cdiff = (C - I)**2
    
    #Within model_2 (unfrozen)
    C2 = (z1norm_2.T @ z2norm_2) / bs 
    cdiff_2 = (C2 - I)**2


    #split projector of unfrozen model up into 3 groups. We will compare each guy to the encoder
    z1norm_21,z1norm_22,z1norm_23,z1norm_24 = z1norm_2[:,0:2048],z1norm_2[:,2048:4096],z1norm_2[:,4096:6144],z1norm_2[:,6144:8192]
    z2norm_21,z2norm_22,z2norm_23,z2norm_24 = z2norm_2[:,0:2048],z2norm_2[:,2048:4096],z2norm_2[:,4096:6144],z2norm_2[:,6144:8192]

    #between model for rr. 
    C_rr11,C_rr12,C_rr13,C_rr14 = cov_mat(z1norm_enc,z1norm_21), cov_mat(z1norm_enc,z1norm_22),cov_mat(z1norm_enc,z1norm_23),cov_mat(z1norm_enc,z1norm_24)
    cdiff_rr_1 = (1/4)*(C_rr11.pow(2) + C_rr12.pow(2) + C_rr13.pow(2) + C_rr14.pow(2))

    C_rr21,C_rr22,C_rr23,C_rr24 = cov_mat(z2norm_enc,z2norm_21), cov_mat(z2norm_enc,z2norm_22),cov_mat(z2norm_enc,z2norm_23),cov_mat(z2norm_enc,z2norm_24)
    cdiff_rr_2 = (1/4)*(C_rr21.pow(2) + C_rr22.pow(2) + C_rr23.pow(2) + C_rr24.pow(2))

    cdiff_rr = 0.5*(cdiff_rr_1 + cdiff_rr_2)

    I_enc = torch.eye(2048).cuda()

    #between model for inv
    C_inva1 = (z1norm.T @ z2norm_2) / bs
    cdiff_inva_1 = (C_inva1+I).pow(2)

    C_inva2 = (z2norm.T @ z1norm_2) / bs
    cdiff_inva_2 = (C_inva2+I).pow(2)

    cdiff_inva = 0.5*(cdiff_inva_1 + cdiff_inva_2)

    loss = (1-t)*(cdiff_2*I).sum() + t*(cdiff_inva*I).sum() + (1-s)*lmb*(cdiff_2*(1-I)).sum() + s*lmb*(cdiff_rr*(1-I_enc)).sum()

    torch.cuda.empty_cache()
    return loss

In [ ]:
#| export

def lf_1_proj(pred,I,lmb,
               t,
               s,
               ):
    "type 1 loss in projector space"

    
    pred1 = pred[0] #frozen
    pred2 = pred[1] #has gradients
    
    
    bs,nf = pred1.size(0)//2,pred1.size(1)

    #All standard, from BT
    z1, z2 = pred1[:bs],pred1[bs:] #so z1 is bs*projection_size, likewise for z2
    z1norm = (z1 - z1.mean(0)) / (z1.std(0, unbiased=False) + 1e-7)
    z2norm = (z2 - z2.mean(0)) / (z2.std(0, unbiased=False) + 1e-7)
    
    z1_2, z2_2 = pred2[:bs],pred2[bs:] #so z1 is bs*projection_size, likewise for z2
    z1norm_2 = (z1_2 - z1_2.mean(0)) / (z1_2.std(0, unbiased=False) + 1e-7)
    z2norm_2 = (z2_2 - z2_2.mean(0)) / (z2_2.std(0, unbiased=False) + 1e-7)
    
    #Make sure gradients are turned off / turned on appropriately
    test_eq(pred1.requires_grad,False)
    test_eq(pred2.requires_grad,True)

    # #Within frozen model 
    # C = (z1norm.T @ z2norm) / bs 
    # cdiff = (C - I)**2
    
    #Within model_2 (unfrozen)
    C2 = (z1norm_2.T @ z2norm_2) / bs 
    cdiff_2 = (C2 - I)**2

    #between model for rr
    C_rr1=(z1norm.T @ z1norm_2) / bs
    cdiff_rr_1 = C_rr1.pow(2)

    C_rr2=(z2norm.T @ z2norm_2) / bs
    cdiff_rr_2 = C_rr2.pow(2)

    cdiff_rr = 0.5*(cdiff_rr_1 + cdiff_rr_2)

    #between model for inv
    C_inva1 = (z1norm.T @ z2norm_2) / bs
    cdiff_inva_1 = (C_inva1+I).pow(2)

    C_inva2 = (z2norm.T @ z1norm_2) / bs
    cdiff_inva_2 = (C_inva2+I).pow(2)

    cdiff_inva = 0.5*(cdiff_inva_1 + cdiff_inva_2)


    loss = (1-t)*(cdiff_2*I).sum() + t*(cdiff_inva*I).sum() + (1-s)*lmb*(cdiff_2*(1-I)).sum() + s*lmb*(cdiff_rr*(1-I)).sum()

    
    torch.cuda.empty_cache()
    return loss

In [ ]:
#| export

def lf_2_enc(pred,I,lmb,
               t,
               s,
               ):
    
    "type 2 loss in encoder space"

    torch.cuda.empty_cache()
    gc.collect()

    pred1 = pred[0] #frozen
    pred2 = pred[1] #has gradients

    pred1_enc = pred[2]
    #pred2_enc = pred[3] #don't actually use this guy
    
    
    bs,nf = pred1.size(0)//2,pred1.size(1)

    #All standard, from BT
    z1, z2 = pred1[:bs],pred1[bs:] #so z1 is bs*projection_size, likewise for z2
    z1norm = (z1 - z1.mean(0)) / (z1.std(0, unbiased=False) + 1e-7)
    z2norm = (z2 - z2.mean(0)) / (z2.std(0, unbiased=False) + 1e-7)
    
    z1_2, z2_2 = pred2[:bs],pred2[bs:] #so z1 is bs*projection_size, likewise for z2
    z1norm_2 = (z1_2 - z1_2.mean(0)) / (z1_2.std(0, unbiased=False) + 1e-7)
    z2norm_2 = (z2_2 - z2_2.mean(0)) / (z2_2.std(0, unbiased=False) + 1e-7)

    
    z1, z2 = pred1_enc[:bs],pred1_enc[bs:] #so z1 is bs*projection_size, likewise for z2
    z1norm_enc = (z1 - z1.mean(0)) / (z1.std(0, unbiased=False) + 1e-7)
    z2norm_enc = (z2 - z2.mean(0)) / (z2.std(0, unbiased=False) + 1e-7)

    #We don't actually need encoder for unfrozen model, I believe, at this point.
    
    #Make sure gradients are turned off / turned on appropriately
    test_eq(pred1.requires_grad,False)
    test_eq(pred2.requires_grad,True)

    # #Within frozen model 
    # C = (z1norm.T @ z2norm) / bs 
    # cdiff = (C - I)**2
    
    #Within model_2 (unfrozen)
    C2 = (z1norm_2.T @ z2norm_2) / bs 
    cdiff_2 = (C2 - I)**2


    #split projector of unfrozen model up into 3 groups. We will compare each guy to the encoder
    z1norm_21,z1norm_22,z1norm_23,z1norm_24 = z1norm_2[:,0:2048],z1norm_2[:,2048:4096],z1norm_2[:,4096:6144],z1norm_2[:,6144:8192]
    z2norm_21,z2norm_22,z2norm_23,z2norm_24 = z2norm_2[:,0:2048],z2norm_2[:,2048:4096],z2norm_2[:,4096:6144],z2norm_2[:,6144:8192]

    #yes, we can make this code nicer

    t1 = z1norm_enc*z1norm_21 #like z^A1 * Z^A2 ...
    t2 = z2norm_enc*z2norm_21
    C_rr_1 = t1.T @ t2 / bs

    t1 = z1norm_enc*z1norm_22 #like z^A1 * Z^A2 ...
    t2 = z2norm_enc*z2norm_22
    C_rr_2 = t1.T @ t2 / bs

    t1 = z1norm_enc*z1norm_23 #like z^A1 * Z^A2 ...
    t2 = z2norm_enc*z2norm_23
    C_rr_3 = t1.T @ t2 / bs

    t1 = z1norm_enc*z1norm_24 #like z^A1 * Z^A2 ...
    t2 = z2norm_enc*z2norm_24
    C_rr_4 = t1.T @ t2 / bs

    cdiff_rr = (1/4)*((C_rr_1.pow(2) + 1e-7).pow(0.5) + (C_rr_2.pow(2) + 1e-7).pow(0.5) + (C_rr_3.pow(2) + 1e-7).pow(0.5) + (C_rr_4.pow(2) + 1e-7).pow(0.5))  #keep the units the same

    I_enc = torch.eye(2048).cuda()

    #between model for inv
    C_inva1 = (z1norm.T @ z2norm_2) / bs
    cdiff_inva_1 = (C_inva1-I).pow(2)

    C_inva2 = (z2norm.T @ z1norm_2) / bs
    cdiff_inva_2 = (C_inva2-I).pow(2)

    cdiff_inva = 0.5*(cdiff_inva_1 + cdiff_inva_2)

    loss = (1-t)*(cdiff_2*I).sum() + t*(cdiff_inva*I).sum() + (1-s)*lmb*(cdiff_2*(1-I)).sum() + s*lmb*(cdiff_rr*(1-I_enc)).sum()
    return loss

In [ ]:
#| export

def lf_2_proj(pred,I,lmb,
               t,
               s,
               ):
    "Type 2 loss in projector space"

    
    pred1 = pred[0] #frozen
    pred2 = pred[1] #has gradients
    
    
    bs,nf = pred1.size(0)//2,pred1.size(1)

    #All standard, from BT
    z1, z2 = pred1[:bs],pred1[bs:] #so z1 is bs*projection_size, likewise for z2
    z1norm = (z1 - z1.mean(0)) / (z1.std(0, unbiased=False) + 1e-7)
    z2norm = (z2 - z2.mean(0)) / (z2.std(0, unbiased=False) + 1e-7)
    
    z1_2, z2_2 = pred2[:bs],pred2[bs:] #so z1 is bs*projection_size, likewise for z2
    z1norm_2 = (z1_2 - z1_2.mean(0)) / (z1_2.std(0, unbiased=False) + 1e-7)
    z2norm_2 = (z2_2 - z2_2.mean(0)) / (z2_2.std(0, unbiased=False) + 1e-7)
    
    #Make sure gradients are turned off / turned on appropriately
    test_eq(pred1.requires_grad,False)
    test_eq(pred2.requires_grad,True)

    # #Within frozen model 
    # C = (z1norm.T @ z2norm) / bs 
    # cdiff = (C - I)**2
    
    #Within model_2 (unfrozen)
    C2 = (z1norm_2.T @ z2norm_2) / bs 
    cdiff_2 = (C2 - I)**2

    #between model for rr
    # C_rr1=(z1norm.T @ z1norm_2) / bs
    # cdiff_rr_1 = C_rr1.pow(2)

    # C_rr2=(z2norm.T @ z2norm_2) / bs
    # cdiff_rr_2 = C_rr2.pow(2)

    # cdiff_rr = 0.5*(cdiff_rr_1 + cdiff_rr_2)

    t1 = z1norm*z1norm_2 #like z^A1 * Z^A2 ...
    t2 = z2norm*z2norm_2

    C_rr = t1.T @ t2 / bs

    cdiff_rr = (C_rr.pow(2) + 1e-7).pow(0.5) #keep the units the same


    #between model for inv
    C_inva1 = (z1norm.T @ z2norm_2) / bs
    cdiff_inva_1 = (C_inva1-I).pow(2)

    C_inva2 = (z2norm.T @ z1norm_2) / bs
    cdiff_inva_2 = (C_inva2-I).pow(2)

    cdiff_inva = 0.5*(cdiff_inva_1 + cdiff_inva_2)


    loss = (1-t)*(cdiff_2*I).sum() + t*(cdiff_inva*I).sum() + (1-s)*lmb*(cdiff_2*(1-I)).sum() + s*lmb*(cdiff_rr*(1-I)).sum()

    
    torch.cuda.empty_cache()
    return loss


In [ ]:
#| export

class main_train_ensemble(main_train):

    def __init__(self,
                 dls_train, #used for training BT (if pretrain=True)
                 dls_tune , #used for tuning
                 dls_valid, #used to compute metrics / evaluate results. 
                 xval, #currently `predict_model` below assumes this is entire validation / test data
                 yval,
                 aug_pipelines, #the aug pipeline for self-supervised learning
                 aug_pipelines_tune, #the aug pipeline for supervised learning
                 aug_pipelines_test, #test (or valid) time augmentations 
                 initial_weights, #Which initial weights to use
                 pretrain, #Whether to fit BT
                 num_epochs, #number of BT fit epochs
                 numfit, #number of tune_fit epochs
                 freeze_num_epochs, #How many epochs to freeze body for when training BT
                 freeze_numfit, #How many epochs to freeze body for when fine tuning
                 ps=8192, #projection size
                 n_in=3, #color channels
                 indim=2048, #dimension output of encoder (2048 for resnet50)
                 outdim=9, #number of classes
                 print_report=False, #F1 metrics etc
                 print_plot=False, #ROC curve
                 tune_model_path=None, #save models after fine tuning
                 model=None, #BT model that has already been trained; i.e. what we are pushing rep away from...
                 ):
        store_attr()
        self.vocab = self.dls_valid.vocab
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        
    def fit(learn,fit_type,epochs,freeze_epochs,initial_weights,pretrain):
        """We can patch in a modification, e.g. if we want subtype of fine_tune:supervised_pretrain to be different
        to fine_tune:bt_pretrain"""

        if fit_type == 'encoder_fine_tune': #i.e. barlow twins

            #learn.encoder_fine_tune(epochs,freeze_epochs=freeze_epochs)
            lr_max=0.0030199517495930195
            print(f'lr_max={lr_max}')
            learn.fit_one_cycle(epochs,lr_max= lr_max)

        elif fit_type == 'fine_tune':

            if pretrain == False:
                print('pretrain was False, and about to fit_one_cycle')
                learn.fit_one_cycle(epochs,lr_max=0.00027542) 

            elif pretrain == True:
                print('pretrain was True, and about to linear_fine_tune')
                learn.linear_fine_tune(epochs,freeze_epochs=freeze_epochs) #This gave very similar performance, when pretrain=False (see above / earlier commit)

            #learn.no_freeze_linear_fine_tune(epochs,freeze_epochs=freeze_epochs) 

        else: raise Exception('Fit policy not of expected form')

    def train_encoder(self):
        "create encoder and (optionally, if pretrain=True) train with BT algorithm, according to fit_policy"


        with torch.no_grad():
            bt_model2,encoder2 = create_ens_model(model=self.model,ps=self.ps,device=self.device)

        if self.pretrain: #train encoder according to fit policy

            lmb=1/8192
            learn = Learner(self.dls_train,bt_model2,cbs=[BarlowTwinsEns(self.aug_pipelines,n_in=self.n_in,lmb=lmb,print_augs=False)])
            main_train_ensemble.fit(learn,fit_type='encoder_fine_tune',
                            epochs=self.num_epochs,freeze_epochs=self.freeze_num_epochs,
                            initial_weights=self.initial_weights,
                            pretrain=self.pretrain
                            )
            
        self.encoder2 = bt_model2.encoder2
        self.bt_model2=bt_model2


    def fine_tune(self):
        "fine tune in supervised fashion, according to tune_fit_policy, and get metrics"

        #encoder = pickle.loads(pickle.dumps(self.encoder)) #We might want to pretrain once and fine tune several times (varying e.g. tune augs)


        model = LM(self.encoder2)
        learn = Learner(self.dls_tune,model,splitter=my_splitter,cbs = [LinearBt(aug_pipelines=self.aug_pipelines_tune,n_in=self.n_in)],wd=0.0)

        #debugging
        #learn = Learner(self.dls_tune,model,cbs = [LinearBt(aug_pipelines=self.aug_pipelines_tune,n_in=self.n_in)],wd=0.0)

        main_train_ensemble.fit(learn,fit_type='fine_tune',
                    epochs=self.numfit,freeze_epochs=self.freeze_numfit,
                    initial_weights=self.initial_weights,
                    pretrain=self.pretrain
                    ) #fine tuning (don't confuse this with fit policy!)
        
        scores,preds, acc = predict_model(self.xval,self.yval,model=model,aug_pipelines_test=self.aug_pipelines_test,numavg=3)
        #metrics dict will have f1 score, auc etc etc
        metrics = classification_report_wrapper(preds, self.yval, self.vocab, print_report=self.print_report)
        auc_dict = plot_roc(self.yval,preds,self.vocab,print_plot=self.print_plot)
        metrics['acc'],metrics['auc_dict'],metrics['scores'],metrics['preds'],metrics['xval'],metrics['yval'] = acc,auc_dict,scores,preds,self.xval,self.yval

        if self.tune_model_path != None:
            metrics['classif_model_path'] = self.tune_model_path
            torch.save(model.state_dict(), self.tune_model_path)


        return metrics 


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()